In [1]:
import numpy as np
import MetaTrader5 as mt5
import pandas as pd
import inspect

from numba import NoneType

In [23]:
mt5.initialize()

True

In [24]:
ac_dict = mt5.account_info()._asdict()

In [25]:
account_size = ac_dict["balance"]
equity = ac_dict["equity"]
leverage = ac_dict["leverage"]

In [29]:
symbol_info = mt5.symbol_info("US100.cash")

In [33]:
s_info = symbol_info._asdict()

tick_value = s_info["trade_tick_value"]
tick_size = s_info["trade_tick_size"]

In [35]:
def trade_pnl(close_price: float,
              open_price: float,
              trade_volume: float,
              tick_value: float,
              tick_size: float,
              direction: str = "buy") -> float:
    """
    Calculate the profit or loss of a trade based on its parameters.

    Args:
        close_price (float): The closing price of the trade.
        open_price (float): The opening price of the trade.
        trade_volume (float): The volume of the trade.
        tick_value (float): The value of a single tick.
        tick_size (float): The size of a single tick.
        direction (str, optional): The trade direction. Either 'buy' or 'sell'.
                                   Defaults to 'buy'.

    Returns:
        float: The profit or loss of the trade.

    Raises:
        ValueError: If the direction is not 'buy' or 'sell'.
    """
    if direction == "buy":
        return (close_price - open_price) * trade_volume * tick_value / tick_size
    elif direction == "sell":
        return (open_price - close_price) * trade_volume * tick_value / tick_size
    else:
        raise ValueError("Direction must be 'buy' or 'sell'")


def get_volume_for_price_change(price_change: float,
                                tick_value: float,
                                tick_size: float,
                                profit_loss: float) -> float:
    """
    Calculate the trade volume required to achieve a specific profit or loss for a given price change.

    Args:
        price_change (float): The change in price.
        tick_value (float): The value of a single tick.
        tick_size (float): The size of a single tick.
        profit_loss (float): The desired profit or loss.

    Returns:
        float: The trade volume required to achieve the specified profit or loss.

    Raises:
        AssertionError: If the price change is not positive.
    """
    assert price_change > 0, "Price change must be positive."
    return profit_loss * tick_size / price_change / tick_value


def get_volume_for_sl(entry_price: float,
                      sl_level: float,
                      direction: str,
                      tick_value: float,
                      profit_loss: float) -> float:
    """
    Calculate the trade volume required to achieve a specific profit or loss
    based on a stop loss level.

    Args:
        entry_price (float): The entry price of the trade.
        sl_level (float): The stop loss level.
        direction (str): The trade direction. Either 'buy' or 'sell'.
        tick_value (float): The value of a single tick.
        profit_loss (float): The desired profit or loss.

    Returns:
        float: The trade volume required to achieve the specified profit or loss.

    Raises:
        AssertionError:
            - If the direction is not 'buy' or 'sell'.
            - If the stop loss level is not valid relative to the entry price.
    """
    assert direction in ["buy", "sell"], "Direction must be either 'buy' or 'sell'."
    if direction == "buy":
        assert sl_level < entry_price, "Stop loss must be below the entry price for a buy trade."
    else:
        assert sl_level > entry_price, "Stop loss must be above the entry price for a sell trade."

    price_change = abs(entry_price - sl_level)
    return get_volume_for_price_change(price_change, tick_value, tick_size, profit_loss)

In [36]:
import os

In [38]:
# List all yamls files
configs = os.listdir("../config/prod")

# Grab all assets along with the strategy name (name of the yaml file without extension)
strategies = [(os.path.splitext(cfg)[0], cfg) for cfg in configs]


In [210]:
import os
import yaml  # Ensure PyYAML is already installed

def list_assets_from_yamls(config_dir: str) -> dict:
    """
    Reads all YAML files in the specified directory and extracts the assets along with their strategy names.

    Args:
        config_dir (str): The path to the directory containing the YAML files.

    Returns:
        dict: A dictionary where the keys are strategy names (file names without extensions),
              and values are the asset data from each YAML file.

    Raises:
        FileNotFoundError: If the specified directory does not exist.
        ValueError: If a YAML file cannot be parsed.
    """
    if not os.path.exists(config_dir):
        raise FileNotFoundError(f"Directory '{config_dir}' does not exist.")

    instances = []
    yaml_files = [f for f in os.listdir(config_dir) if f.endswith(".yaml")]

    for yaml_file in yaml_files:
        strategy_name = os.path.splitext(yaml_file)[0]  # Get the file name without extension
        yaml_path = os.path.join(config_dir, yaml_file)

        with open(yaml_path, 'r') as file:
            try:
                data = yaml.safe_load(file)  # Safely load the YAML data
                instances += [(i["strategy_type"], i["symbols"][0]) for i in data.values()]

            except yaml.YAMLError as e:
                raise ValueError(f"Error reading YAML file '{yaml_file}': {e}")

    return instances

In [61]:
config_path     = "../config/prod"  # Path to the directory containing YAML files
instances       = list_assets_from_yamls(config_path)
instances       = pd.DataFrame(instances, columns=["strategy_type", "symbol"])
symbols_running = instances["symbol"].unique()

In [109]:
import datetime
import matplotlib.pyplot as plt

In [221]:
def recover_price_diff(symbol):
    today = datetime.datetime.utcnow()
    last_month = today - datetime.timedelta(days=92)
    rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_M15, last_month, today)
    rates = pd.DataFrame(rates)
    rates.index = pd.to_datetime(rates.time, unit="s")

    symbol_info = mt5.symbol_info(symbol)
    tick_value = symbol_info.trade_tick_value
    tick_size = symbol_info.trade_tick_size
    price_diffs = rates.close.diff() * tick_value / tick_size

    return price_diffs

def compute_symbols_daily_cov(symbols_running):
    price_diff_df = pd.DataFrame()
    for symbol in symbols_running:
        price_diff_df[symbol] = recover_price_diff(symbol)
    # Number of 15-minute bars in one day
    return price_diff_df.cov() * 4 * 24

In [195]:
def fetch_daily_volatility(symbol):
    today = datetime.datetime.utcnow()
    last_month = today - datetime.timedelta(days=92)
    rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_M15, last_month, today)
    rates = pd.DataFrame(rates)
    rates.index = pd.to_datetime(rates.time, unit="s")

    symbol_info = mt5.symbol_info(symbol)
    tick_value = symbol_info.trade_tick_value
    tick_size = symbol_info.trade_tick_size
    price_diffs = rates.close.diff() * tick_value / tick_size

    try:
        daily_vols = price_diffs.resample("D").apply(lambda x: np.sum(np.square(x))).dropna()
        return daily_vols.ewm(span=5).mean().iloc[-10:]
    except:
        return np.nan

In [198]:
daily_vols_series = list(map(fetch_daily_volatility, symbols_running))
daily_vols_series = pd.DataFrame(daily_vols_series, index=symbols_running)

In [338]:
strategy_weights = dict()

strategy_weights["metafvg"] = 20/np.sqrt(48)
strategy_weights["metago"] = np.sqrt(1.5)
strategy_weights["metane"] = 10/np.sqrt(12)
strategy_weights["metaga"] = 15/np.sqrt(48)

running_strategies = list(strategy_weights.keys())
symbols_running_cov = compute_symbols_daily_cov(symbols_running)

In [339]:
running_instances = instances[instances["strategy_type"].isin(running_strategies)]

In [340]:
historic_deals = pd.read_csv("merged_deals.csv")

historic_deals["date"] = historic_deals["time_close"].apply(lambda x: pd.to_datetime(x).date())
historic_deals["type"] = historic_deals["comment_open"].apply(lambda x: x.split("_")[0])
num_trades = historic_deals[["date", "comment_open"]].groupby(["date", "comment_open"]).apply(len)
num_trades.reset_index().pivot(index="date", columns="comment_open", values=0).mean()

C:\Users\Trismegist\AppData\Local\Temp\ipykernel_14980\2132112352.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  num_trades = historic_deals[["date", "comment_open"]].groupby(["date", "comment_open"]).apply(len)


comment_open
metafvg_audjpy     9.200000
metafvg_btc        5.000000
metafvg_dax       19.857143
metafvg_eurusd    19.250000
metafvg_nas       15.842105
metaga_f1         18.741935
metaga_f2         16.250000
metaga_i1         22.030303
metaga_i2         19.354839
metago_f1          1.545455
metago_f10         1.333333
metago_f2          1.285714
metago_f4          1.555556
metago_f5          1.444444
metago_f6          1.000000
metago_f7          1.166667
metago_f8          1.250000
metago_f9          1.000000
metago_i2          1.400000
metane_audjpy      7.500000
metane_dax         9.000000
metane_eurusd      1.000000
metane_nas         1.000000
dtype: float64

In [341]:
num_trades

date        comment_open
2025-08-05  metafvg_nas      3
2025-08-06  metafvg_nas     18
2025-08-07  metafvg_nas      7
2025-08-08  metafvg_nas      2
2025-08-09  metafvg_btc      1
                            ..
2025-09-26  metago_f4        2
            metago_f5        2
            metago_f7        1
            metago_f8        2
            metago_f9        1
Length: 260, dtype: int64

In [342]:
# def fetch_strategies_cov(running_instances, symbols_cov, strategy_weights):
strategies_running_cov = pd.DataFrame()

for f_instance in running_instances.itertuples():
    for s_instance in running_instances.itertuples():
        if f_instance[1] != s_instance[1]:
            val = 0
        else:
            f_symbol = f_instance[2]
            s_symbol = s_instance[2]

            val = symbols_running_cov.loc[f_symbol, s_symbol] * strategy_weights[f_instance[1]]**2

        strategies_running_cov.loc[f_instance, s_instance] = val

In [343]:
import cvxpy as cp

In [344]:
w = cp.Variable(len(running_instances))
Sig = strategies_running_cov.values
Sig = Sig + 1e-3 * np.eye(len(Sig))

In [345]:
# Ensure Sig is symmetric
assert np.allclose(Sig, Sig.T), "Covariance matrix not symmetric"

# Check positive semidefinite
eigenvalues = np.linalg.eigvals(Sig)
assert np.all(eigenvalues >= -1e-8), "Matrix not positive semidefinite"

In [346]:
np.linalg.eigvals(Sig)

array([3.80470478e+07, 2.21365088e+06, 4.92471358e+05, 1.77529379e+05,
       1.26226714e+06, 6.82670330e+05, 2.44760346e+05, 1.02576899e+05,
       2.12108788e+06, 7.22752243e+05, 5.44719862e+05, 1.94290937e+05,
       1.06729999e+05, 6.34990701e+04, 4.15059145e+04, 1.00000005e-03,
       1.00000005e-03, 9.99999996e-04, 1.00000000e-03, 3.80470478e+07,
       2.21365088e+06, 4.92471358e+05, 1.77529379e+05])

In [359]:
# If Sig is positive semidefinite, use Cholesky decomposition
w = cp.Variable(len(Sig))
sigma_diag = np.sqrt(np.diag(Sig))

prob = cp.Problem(
    cp.Minimize(cp.quad_form(w, Sig)),
    [cp.sum(cp.multiply(w, sigma_diag)) == 5e2,
     w >= 0],
)
prob.solve()

11368.347642776134

In [360]:
print(f"w shape: {w.shape}")
print(f"Sig shape: {Sig.shape}")

w shape: (23,)
Sig shape: (23, 23)


In [361]:
pd.Series(w.value, index=running_instances)

(metafvg, BTCUSD)        0.002260
(metafvg, EURUSD)        0.012991
(metafvg, GER40.cash)    0.021570
(metafvg, US100.cash)    0.011619
(metaga, EURUSD)         0.018947
(metaga, AUDJPY)         0.021673
(metaga, GER40.cash)     0.025696
(metaga, US100.cash)     0.016202
(metago, EURUSD)         0.111397
(metago, USDJPY)         0.055573
(metago, USDCHF)         0.114556
(metago, USDCAD)         0.099208
(metago, EURHUF)         0.011916
(metago, USDCNH)         0.062544
(metago, USDZAR)         0.011890
(metago, EURUSD)         0.111397
(metago, USDZAR)         0.011890
(metago, USDCNH)         0.062544
(metago, EURHUF)         0.011916
(metane, BTCUSD)         0.002260
(metane, EURUSD)         0.012991
(metane, GER40.cash)     0.021570
(metane, US100.cash)     0.011619
dtype: float64

In [364]:
np.sum(np.round(w.value, 2) @ sigma_diag)

455.3339713842126

In [365]:
np.round(w.value, 2)

array([0.  , 0.01, 0.02, 0.01, 0.02, 0.02, 0.03, 0.02, 0.11, 0.06, 0.11,
       0.1 , 0.01, 0.06, 0.01, 0.11, 0.01, 0.06, 0.01, 0.  , 0.01, 0.02,
       0.01])

In [ ]:
from metalib.metastrategy import MetaStrategy
import pytz
from datetime import datetime, timedelta

class MetaScale(MetaStrategy):
    """MetaTrader FVG (Fair Value Gap) Trading Strategy"""

    def __init__(self,
                 tag,
                 ):
        """
        Initialize the Position Sizing Strategy
        """
        super().__init__([], 0, tag) # Init with empty symbols and timeframe
        print(f"{self.tag}::    Initializing MetaScaler strategy..")

    def _pull_yaml_configs(self, config_dir: str) -> NoneType:
        """
        Reads all YAML files in the specified directory and extracts the assets along with their strategy names.

        Args:
            config_dir (str): The path to the directory containing the YAML files.

        Returns:
            dict: A dictionary where the keys are strategy names (file names without extensions),
                  and values are the asset data from each YAML file.

        Raises:
            FileNotFoundError: If the specified directory does not exist.
            ValueError: If a YAML file cannot be parsed.
        """

        if not os.path.exists(config_dir):
            raise FileNotFoundError(f"Directory '{config_dir}' does not exist.")

        instances = []
        yaml_files = [f for f in os.listdir(config_dir) if f.endswith(".yaml")]

        for yaml_file in yaml_files:
            strategy_name = os.path.splitext(yaml_file)[0]  # Get the file name without extension
            yaml_path = os.path.join(config_dir, yaml_file)

            with open(yaml_path, 'r') as file:
                try:
                    data = yaml.safe_load(file)  # Safely load the YAML data
                    instances += [(i["strategy_type"], i["symbols"][0]) for i in data.values()]

                except yaml.YAMLError as e:
                    raise ValueError(f"Error reading YAML file '{yaml_file}': {e}")

        instances = pd.DataFrame(instances, columns=["strategy_type", "symbol"])
        self.instances = instances
        self.symbols_running = instances["symbol"].unique()
        return

    def _compute_symbols_daily_cov(self, symbols_running):
        price_diff_df = pd.DataFrame()
        for symbol in symbols_running:
            price_diff_df[symbol] = recover_price_diff(symbol)
        # Number of 15-minute bars in one day
        return price_diff_df.cov() * 4 * 24

    def _apply_mt5_tick_params(self, symbol, ):
        symbol_info = mt5.symbol_info(symbol)
        tick_value = symbol_info.trade_tick_value
        tick_size = symbol_info.trade_tick_size
        return tick_value, tick_size

    def fit(self):
        print(f"{self.tag}::    Starting the MetaScaler fit!!..")
        # Pulling YAML configs
        self._pull_yaml_configs("../config/prod")
        print(f"{self.tag}::    Strategy instances: {self.instances}")
        print(f"{self.tag}::    Running symbols: {self.symbols_running}")

        # Load the last 92 days of data
        utc = pytz.timezone('UTC')
        # Get the current time in UTC
        end_time = datetime.now(utc)
        start_time = end_time - timedelta(days=92)
        # Set the time components to 0 (midnight) and maintain the timezone
        end_time = end_time.replace(hour=0, minute=0, second=0, microsecond=0).astimezone(utc)
        start_time = start_time.astimezone(utc)

        # Pulling last days of data
        self.loadData(start_time, end_time)
        data = pd.concat([self.data[s].close for s in self.symbols], axis=1)
        data.columns = self.symbols_running

        # Convert using mt5 tick value and size
        symbol_info = mt5.symbol_info(symbol)
        tick_value = symbol_info.trade_tick_value
        tick_size = symbol_info.trade_tick_size
        price_diffs = data.diff() * tick_value / tick_size

        # Compute daily covariance of price differences
        # Apply strategy weigthing for each strategy
        # Compute the covariance of strategies in a block-wise manner

